In [1]:
!unzip "C:/Users/Dharshini/Downloads/archive.zip"

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [15]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten 
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
import cv2
import numpy as np
import os
import pandas as pd

In [16]:
# VGG model requires 224*224 input so we are going to re-size all images
IMAGE_SIZE = [224, 224]

In [17]:
! mkdir image

A subdirectory or file image already exists.


In [18]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 9A0C-0B07

 Directory of c:\Users\Dharshini\Downloads

16-01-2025  12:15    <DIR>          .
16-01-2025  11:57    <DIR>          ..
06-12-2024  09:22    <DIR>          .idea
05-12-2024  19:34        22,573,639 20190614_133539_IR_H264.MOV
05-12-2024  19:06         3,859,919 20190614_133539_IR_H264-export.json
04-01-2025  21:57            44,776 4675520024.pdf
04-01-2025  22:06           102,843 4d9bc6aa-0dd1-4cf0-8d59-e44e8779df0c.pdf
15-01-2025  19:11     2,000,605,556 archive.zip
16-01-2025  12:12            38,485 binary-classification-of-image.ipynb
15-01-2025  18:56       159,304,728 bio.h5
03-12-2024  14:00            90,231 BT_HR_Non_Disclosure_Agreement_2023.docx
03-12-2024  14:06            69,351 BT_HR_Personnel Information Form.xlsx
02-12-2024  18:15        10,384,768 ChromeSetup.exe
02-12-2024  18:15               577 debug.log
23-12-2024  19:09    <DIR>          demo
24-12-2024  10:20            15,122 demo (1).zip
2

In [19]:
! mv TRAIN.1/ image/

'mv' is not recognized as an internal or external command,
operable program or batch file.


In [20]:
! mv TEST/ image/

'mv' is not recognized as an internal or external command,
operable program or batch file.


In [21]:
file_path="C:/Users/Dharshini/Downloads"

In [22]:
# example of progressively loading images from file
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# create generator
datagen = ImageDataGenerator(rescale = 1./255,
                             shear_range = 0.2,
                             zoom_range = 0.2,
                             horizontal_flip = True,
                             vertical_flip=True,
                             rotation_range=30)
# prepare an iterators for each dataset
train_it = datagen.flow_from_directory('C:\\Users\\Dharshini\\Downloads\\TRAIN.1', 
                                       class_mode='categorical',
                                       target_size=(224, 224),
                                       batch_size=2,
                                       seed=7)

test_it = datagen.flow_from_directory('C:\\Users\\Dharshini\\Downloads\\TEST',  
                                       class_mode='categorical',
                                       target_size=(224, 224),
                                       batch_size=2,
                                       seed=7)

Found 59922 images belonging to 2 classes.
Found 16726 images belonging to 2 classes.


In [23]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras import Sequential


## Using CNN

In [24]:
model = Sequential()
model.add(Conv2D(32, (5,5),padding='valid',activation='relu',input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(2,activation='softmax'))

c:\Users\Dharshini\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
from tensorflow.keras.losses import CategoricalCrossentropy
model.compile(
  optimizer='adam',
  loss=CategoricalCrossentropy(),
  metrics=['accuracy'])

In [28]:
history = model.fit( 
  train_it,
  validation_data=test_it,
  epochs=5)

Epoch 1/5
29961/29961 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step - accuracy: 0.8231 - loss: 0.4230

KeyboardInterrupt: 

In [10]:
model.save("model.h5")

c:\Users\pvtes\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Using VGG16 model

In [ ]:
# VGG16 model

In [11]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [12]:
for layer in vgg.layers:
  layer.trainable = False

In [13]:
output_classes = 2

In [14]:
##Adding flatten and Dense Layer
x = Flatten()(vgg.output)
x = Dense(1000, activation='relu')(x)

prediction = Dense(output_classes, activation='softmax')(x)

In [15]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [ ]:
# view the structure of the model
model.summary()

In [16]:
# tell the model what cost and optimization method to use
from tensorflow.keras.losses import CategoricalCrossentropy
model.compile(
  loss=CategoricalCrossentropy(),
  optimizer='adam',
  metrics=['accuracy']
)

In [20]:
history_vgg= model.fit(
  train_it,
  validation_data=test_it,
  epochs=5 # keep epochs=5 if you want to just check as it take more computational time
)

Epoch 1/5
29961/29961 [==============================] - 7421s 248ms/step - loss: 0.1986 - accuracy: 0.9253 - val_loss: 0.3561 - val_accuracy: 0.8236
Epoch 2/5
29961/29961 [==============================] - 7420s 248ms/step - loss: 0.1619 - accuracy: 0.9382 - val_loss: 0.2624 - val_accuracy: 0.8884
Epoch 3/5
29961/29961 [==============================] - ETA: 0s - loss: 0.1571 - accuracy: 0.9405

KeyboardInterrupt: 

In [22]:
##SAVING A MODEL

In [23]:
model.save("bio.h5")

##If model is trained already,Then  Load model

In [2]:
from keras.models import load_model
 
# load model
model = load_model('bio.h5')

## Checking model with external image

In [4]:

from keras.preprocessing.image import load_img
image = load_img("D:\\archive\\image\\image.jpg", target_size=(224, 224))
img = np.array(image)
img = img / 255.0
img = img.reshape(1,224,224,3)
label = model.predict(img)

1/1 [==============================] - 0s 74ms/step


In [5]:
l={"Biodegradable":label[0][0],"NON-Biodegradable":label[0][1]}
def get_key(val):
    for key, value in l.items():
         if val == value:
             return key
 
    return "key doesn't exist"
#label[0][1]

In [9]:
check=label.max()
get_key(check)

'NON-Biodegradable'